In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from boruta import BorutaPy as bp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
import time
from sklearn.preprocessing import MinMaxScaler

In [10]:
data = pd.read_csv('specPhotoDR10v2_GauthamG_0.csv')

In [11]:
data.dropna(inplace = True)

In [12]:
scaler = MinMaxScaler()
data[data.columns] = scaler.fit_transform(data[data.columns])
data.head()

,specObjID,objid,ra,dec,photoz,photoz_err,specz,specz_err,dered_u,dered_g,...,expPhi_g,expAB_r,expRad_r,expPhi_r,expAB_i,expRad_i,expPhi_i,expAB_z,expRad_z,expPhi_z
0,0.081865,0.002829,0.154768,0.128969,0.297002,0.999681,0.043863,0.001021,0.533379,0.480263,...,0.464509,0.824348,0.013380,0.479943,0.796099,0.013494,0.480844,0.606807,0.010235,0.506055
1,0.081868,0.002829,0.156205,0.129019,0.584260,0.999817,0.064740,0.001021,0.479019,0.490616,...,0.540118,0.594466,0.013163,0.515143,0.610589,0.013029,0.511761,0.279182,0.012368,0.545185
2,0.081493,0.002877,0.141090,0.136944,0.152211,0.999683,0.022515,0.001021,0.428546,0.415922,...,0.475371,0.913257,0.035797,0.476118,0.890801,0.034961,0.484317,0.949539,0.025079,0.470135
3,0.081494,0.002877,0.141090,0.136944,0.152211,0.999683,0.022520,0.001021,0.428546,0.415922,...,0.475371,0.913257,0.035797,0.476118,0.890801,0.034961,0.484317,0.949539,0.025079,0.470135
6,0.081493,0.002877,0.141406,0.138006,0.606950,0.999776,0.001497,0.001021,0.600054,0.507319,...,0.578584,0.172073,0.025074,0.583177,0.338253,0.005512,0.543976,0.197601,0.006683,0.507553


In [13]:
data.dropna(inplace = True)

In [14]:
data = data.drop(['specObjID', 'objid'], axis = 1)

In [15]:
XErr = []
sub1 = 'err'
for a in list(data.columns):
  if sub1 in a:
    XErr.append(a)
sub2 = 'Err'
for a in list(data.columns):
  if sub2 in a:
    XErr.append(a)

In [16]:
data = data.drop(XErr, axis = 1)

In [17]:
data = data.drop(['specType', 'photpType'], axis = 1)

In [18]:
df = data

In [19]:
RFR = RandomForestRegressor(n_jobs = 4, oob_score = True)
X = np.array(data.drop(['photoz'], axis = 1))
y = np.array(data['photoz'])

In [20]:
boruta_selector = bp(RFR, n_estimators='auto', verbose=2)

In [21]:
boruta_selector.fit(X, y)
selected_features = [df.columns[i] for i, x in enumerate(feat_selector.support_) if x]
print(selected_features)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	41
Tentative: 	0
Rejected: 	7


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	41
Tentative: 	0
Rejected: 	7


NameError: name 'feat_selector' is not defined

In [22]:
print(boruta_selector.n_features_)

41


In [23]:
selected_features = [df.columns[i] for i, x in enumerate(boruta_selector.feat_selector.support_) if x]
print(selected_features)

AttributeError: 'BorutaPy' object has no attribute 'feat_selector'

In [30]:
(boruta_selector.ranking_).shape

(48,)

In [28]:
p = 0
for a in boruta_selector.ranking_:
    if a == 1:
        p += 1

In [33]:
feature_df = pd.DataFrame(data.drop(['photoz'], axis=1).columns.tolist(), columns=['features'])

In [34]:
feature_df['rank']=boruta_selector.ranking_

In [35]:
feature_df = feature_df.sort_values('rank', ascending=True).reset_index(drop=True)
print ('\n Top %d features:' % boruta_selector.n_features_)
print (feature_df.head(boruta_selector.n_features_))
feature_df.to_csv('boruta-feature-ranking.csv', index=False)


 Top 41 features:
      features  rank
0           ra     1
1     psfMag_r     1
2     psfMag_i     1
3     psfMag_z     1
4   fiberMag_u     1
5   fiberMag_g     1
6   fiberMag_r     1
7   fiberMag_i     1
8   fiberMag_z     1
9      expAB_u     1
10    expRad_u     1
11    expPhi_u     1
12     expAB_g     1
13    expRad_g     1
14    expPhi_g     1
15     expAB_r     1
16    expRad_r     1
17     expAB_i     1
18    expRad_i     1
19     expAB_z     1
20    psfMag_g     1
21    expRad_z     1
22    psfMag_u     1
23  petroRad_g     1
24         dec     1
25       specz     1
26     dered_u     1
27     dered_g     1
28     dered_r     1
29     dered_i     1
30     dered_z     1
31  petroRad_u     1
32  petroRad_r     1
33    expPhi_z     1
34  petroRad_z     1
35    deVRad_u     1
36    deVRad_g     1
37    deVRad_r     1
38    deVRad_i     1
39    deVRad_z     1
40  petroRad_i     1


In [143]:
# Feat_imp through RF and DT(The same thing ~ inherently use gini importance)

In [69]:
data = pd.read_csv('specPhotoDR10v2_GauthamG_0.csv')

In [71]:
data.dropna(inplace = True)

In [72]:
scaler = MinMaxScaler()
data[data.columns] = scaler.fit_transform(data[data.columns])
data.head()

,specObjID,objid,ra,dec,photoz,photoz_err,specz,specz_err,dered_u,dered_g,...,expPhi_g,expAB_r,expRad_r,expPhi_r,expAB_i,expRad_i,expPhi_i,expAB_z,expRad_z,expPhi_z
0,0.081865,0.002829,0.154768,0.128969,0.297002,0.999681,0.043863,0.001021,0.533379,0.480263,...,0.464509,0.824348,0.013380,0.479943,0.796099,0.013494,0.480844,0.606807,0.010235,0.506055
1,0.081868,0.002829,0.156205,0.129019,0.584260,0.999817,0.064740,0.001021,0.479019,0.490616,...,0.540118,0.594466,0.013163,0.515143,0.610589,0.013029,0.511761,0.279182,0.012368,0.545185
2,0.081493,0.002877,0.141090,0.136944,0.152211,0.999683,0.022515,0.001021,0.428546,0.415922,...,0.475371,0.913257,0.035797,0.476118,0.890801,0.034961,0.484317,0.949539,0.025079,0.470135
3,0.081494,0.002877,0.141090,0.136944,0.152211,0.999683,0.022520,0.001021,0.428546,0.415922,...,0.475371,0.913257,0.035797,0.476118,0.890801,0.034961,0.484317,0.949539,0.025079,0.470135
6,0.081493,0.002877,0.141406,0.138006,0.606950,0.999776,0.001497,0.001021,0.600054,0.507319,...,0.578584,0.172073,0.025074,0.583177,0.338253,0.005512,0.543976,0.197601,0.006683,0.507553


In [73]:
data.dropna(inplace = True)

In [74]:
data = data.drop(['specObjID', 'objid'], axis = 1)

In [98]:
XErr = []
sub1 = 'err'
for a in list(data.columns):
  if sub1 in a:
    XErr.append(a)
sub2 = 'Err'
for a in list(data.columns):
  if sub2 in a:
    XErr.append(a)

In [99]:
data = data.drop(XErr, axis = 1)

In [100]:
data.drop(['specz'], axis = 1)

,ra,dec,photoz,dered_u,dered_g,dered_r,dered_i,dered_z,specType,photpType,...,expPhi_g,expAB_r,expRad_r,expPhi_r,expAB_i,expRad_i,expPhi_i,expAB_z,expRad_z,expPhi_z
0,0.154768,0.128969,0.297002,0.533379,0.480263,0.430524,0.430273,0.458437,0.0,0.0,...,0.464509,0.824348,0.013380,0.479943,0.796099,0.013494,0.480844,0.606807,0.010235,0.506055
1,0.156205,0.129019,0.584260,0.479019,0.490616,0.486457,0.483850,0.509339,0.0,0.0,...,0.540118,0.594466,0.013163,0.515143,0.610589,0.013029,0.511761,0.279182,0.012368,0.545185
2,0.141090,0.136944,0.152211,0.428546,0.415922,0.396063,0.403649,0.433152,0.0,0.0,...,0.475371,0.913257,0.035797,0.476118,0.890801,0.034961,0.484317,0.949539,0.025079,0.470135
3,0.141090,0.136944,0.152211,0.428546,0.415922,0.396063,0.403649,0.433152,0.0,0.0,...,0.475371,0.913257,0.035797,0.476118,0.890801,0.034961,0.484317,0.949539,0.025079,0.470135
6,0.141406,0.138006,0.606950,0.600054,0.507319,0.476039,0.449363,0.459617,0.0,0.0,...,0.578584,0.172073,0.025074,0.583177,0.338253,0.005512,0.543976,0.197601,0.006683,0.507553
7,0.141406,0.138006,0.606950,0.600054,0.507319,0.476039,0.449363,0.459617,0.0,0.0,...,0.578584,0.172073,0.025074,0.583177,0.338253,0.005512,0.543976,0.197601,0.006683,0.507553
8,0.141024,0.136797,0.215520,0.499150,0.492537,0.489016,0.506216,0.530430,0.0,0.0,...,0.469954,0.762583,0.015317,0.510007,0.848450,0.014165,0.522052,0.944101,0.009391,0.555469
9,0.141024,0.136797,0.215520,0.499150,0.492537,0.489016,0.506216,0.530430,0.0,0.0,...,0.469954,0.762583,0.015317,0.510007,0.848450,0.014165,0.522052,0.944101,0.009391,0.555469
12,0.141219,0.137536,0.272046,0.481481,0.472381,0.459136,0.470366,0.491593,0.0,0.0,...,0.371871,0.602598,0.020707,0.355312,0.684223,0.019705,0.387830,0.634482,0.016690,0.374273
13,0.141219,0.137536,0.272046,0.481481,0.472381,0.459136,0.470366,0.491593,0.0,0.0,...,0.371871,0.602598,0.020707,0.355312,0.684223,0.019705,0.387830,0.634482,0.016690,0.374273


In [106]:
# for p in data.columns:
#     print(p)
data = data.drop(['specType', 'photpType'], axis = 1)
for p in data.columns:
    print(p)

ra
dec
photoz
dered_u
dered_g
dered_r
dered_i
dered_z
petroRad_u
petroRad_g
petroRad_r
petroRad_i
petroRad_z
deVRad_u
deVRad_g
deVRad_r
deVRad_i
deVRad_z
extinction_u
extinction_g
extinction_r
extinction_i
extinction_z
psfMag_u
psfMag_g
psfMag_r
psfMag_i
psfMag_z
fiberMag_u
fiberMag_g
fiberMag_r
fiberMag_i
fiberMag_z
expAB_u
expRad_u
expPhi_u
expAB_g
expRad_g
expPhi_g
expAB_r
expRad_r
expPhi_r
expAB_i
expRad_i
expPhi_i
expAB_z
expRad_z
expPhi_z


In [107]:
RFR = RandomForestRegressor()
X = data.drop(['photoz'], axis = 1)
y = data['photoz']
names = list(X.columns)

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

In [109]:
DTR = DecisionTreeRegressor()

In [110]:
DTR.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [111]:
pred = DTR.predict(X_test)
DTR.score(X_test, y_test)

0.9554561082156433

In [112]:
B = (sorted(zip(map(lambda x: round(x, 4), DTR.feature_importances_), names), reverse=True))

In [113]:
i = 0
fnames = []
for b in B:
    if i<20:
        fnames.append(b)
        i += 1
    else:
        i += 1
np.array(fnames).shape

(20, 2)

In [114]:
fnames

[(0.8244, 'dered_g'),
 (0.0803, 'fiberMag_r'),
 (0.0114, 'fiberMag_i'),
 (0.0114, 'dered_u'),
 (0.0092, 'psfMag_z'),
 (0.0068, 'psfMag_g'),
 (0.0061, 'fiberMag_u'),
 (0.006, 'fiberMag_z'),
 (0.0059, 'fiberMag_g'),
 (0.0055, 'dered_i'),
 (0.0049, 'dered_r'),
 (0.0044, 'dered_z'),
 (0.0022, 'expAB_r'),
 (0.002, 'psfMag_u'),
 (0.002, 'psfMag_r'),
 (0.002, 'deVRad_g'),
 (0.0013, 'psfMag_i'),
 (0.0009, 'deVRad_z'),
 (0.0008, 'expRad_u'),
 (0.0008, 'expAB_i')]

In [115]:
rfr = RandomForestRegressor()

In [116]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [117]:
# pred1 = DTR.predict(X_test)
rfr.score(X_test, y_test)

0.9773391089565916

In [118]:
A = (sorted(zip(map(lambda x: round(x, 4), rfr.feature_importances_), names), reverse=True))

In [119]:
i = 0
fnames = []
for a in A:
    if i<20:
        fnames.append(a)
        i += 1
    else:
        i += 1
np.array(fnames).shape

(20, 2)

In [120]:
fnames

[(0.8231, 'dered_g'),
 (0.0811, 'fiberMag_r'),
 (0.0121, 'dered_u'),
 (0.0109, 'fiberMag_i'),
 (0.0083, 'psfMag_z'),
 (0.0068, 'psfMag_g'),
 (0.0067, 'fiberMag_g'),
 (0.006, 'fiberMag_z'),
 (0.0054, 'fiberMag_u'),
 (0.0054, 'dered_i'),
 (0.0049, 'dered_z'),
 (0.0049, 'dered_r'),
 (0.0021, 'deVRad_g'),
 (0.002, 'expAB_r'),
 (0.0019, 'psfMag_r'),
 (0.0018, 'psfMag_u'),
 (0.0015, 'psfMag_i'),
 (0.0011, 'expRad_u'),
 (0.0009, 'deVRad_z'),
 (0.0008, 'expAB_z')]

In [122]:
data1_columns = []
for (b, c) in fnames:
    data1_columns.append(c)

In [125]:
data1_columns.append('photoz')

In [127]:
data1 = data[data1_columns]

In [144]:
print("Top 20 Features - Gini Importance")
for a in data1.columns:
    print(a)

Top 20 Features - Gini Importance
dered_g
fiberMag_r
dered_u
fiberMag_i
psfMag_z
psfMag_g
fiberMag_g
fiberMag_z
fiberMag_u
dered_i
dered_z
dered_r
deVRad_g
expAB_r
psfMag_r
psfMag_u
psfMag_i
expRad_u
deVRad_z
expAB_z
photoz
